In [3]:
# Imports
import polars as pl 
import os

print(os.getcwd())



/mnt/c/econ412/learn_polars


In [6]:
os.chdir("/mnt/c/ECON412/class_project/ECON_412_Project_alw_fork")
print(os.getcwd())

/mnt/c/ECON412/class_project/ECON_412_Project_alw_fork


In [13]:
# Sample file to explore
filename = "Data/Ticker_Data/MMM.csv"

df_test = pl.read_csv_batched(filename)
batches = df_test.next_batches(1)
batches[0].schema

Schema([('Price', String),
        ('Close', String),
        ('High', String),
        ('Low', String),
        ('Open', String),
        ('Volume', String)])